# Machine Learning - Practical work 01

## 3. Quick numpy introduction

### Max of each row and max of each column

In [28]:
import numpy as np

array1 = np.array([
    [1, 2, 3],
    [4, 5, 6]
])

print("The max of each row is {}".format(np.amax(array1, axis = 1)))
print("The max of each column is {}".format(np.amax(array1, axis = 0)))

The max of each row is [3 6]
The max of each column is [4 5 6]


### Print the elements of array1 that are less than 4

In [30]:
import numpy as np

array1 = np.array([
    [1, 2, 3],
    [4, 5, 6]
])

mask = array1 < 4
print("Elements of array1 that are < 4: {}".format(array1[mask]))

Elements of array1 that are < 4: [1 2 3]
